In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
import fbprophet
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Load and prepare data
df = pd.read_csv('amazon seller.csv')

if 'date' not in df.columns:
    start_date = '2023-01-01'
    df['date'] = pd.date_range(start=start_date, periods=len(df), freq='D')

df.set_index('date', inplace=True)

if 'sales' not in df.columns:
    np.random.seed(0)
    df['sales'] = np.random.randint(50, 500, size=len(df))

df.ffill(inplace=True)
df_resampled = df['sales'].resample('M').sum()

# Normalize data for LSTM
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df_resampled.values.reshape(-1, 1))

# Prepare training and test data
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# ARIMA Model
def arima_model(train, test):
    model = ARIMA(train, order=(1,1,1))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=len(test))
    return forecast, model_fit

arima_forecast, arima_fit = arima_model(df_resampled[:train_size], df_resampled[train_size:])

# Prophet Model
def prophet_model(df):
    df_prophet = df.reset_index().rename(columns={'date': 'ds', 'sales': 'y'})
    model = fbprophet.Prophet()
    model.fit(df_prophet)
    future = model.make_future_dataframe(periods=len(df_resampled) - train_size, freq='M')
    forecast = model.predict(future)
    return forecast

prophet_forecast = prophet_model(df)

# LSTM Model
def create_generator(data, n_input, n_output):
    return TimeseriesGenerator(data, data, length=n_input, batch_size=1)

def lstm_model(train_data, test_data, n_input=12):
    train_generator = create_generator(train_data, n_input, 1)
    test_generator = create_generator(test_data, n_input, 1)

    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_input, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    model.fit(train_generator, epochs=50, verbose=0)
    
    predictions = []
    for i in range(len(test_generator)):
        x, _ = test_generator[i]
        y_pred = model.predict(x)
        predictions.append(y_pred[0, 0])
    
    return scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

lstm_forecast = lstm_model(train_data, test_data)

# Inverse transform Prophet forecasts
prophet_forecast = prophet_forecast[['ds', 'yhat']].set_index('ds').loc[df_resampled.index[-len(test_data):]]
prophet_forecast = scaler.inverse_transform(prophet_forecast['yhat'].values.reshape(-1, 1))

# Evaluate performance
def evaluate_performance(true_values, forecasts):
    rmse = np.sqrt(mean_squared_error(true_values, forecasts))
    return rmse

test_dates = df_resampled.index[-len(test_data):]
arima_rmse = evaluate_performance(df_resampled[-len(test_data):], arima_forecast)
prophet_rmse = evaluate_performance(df_resampled[-len(test_data):], prophet_forecast)
lstm_rmse = evaluate_performance(df_resampled[-len(test_data):], lstm_forecast)

print(f'ARIMA RMSE: {arima_rmse}')
print(f'Prophet RMSE: {prophet_rmse}')
print(f'LSTM RMSE: {lstm_rmse}')

# Plot results
plt.figure(figsize=(15,10))

# Plot ARIMA
plt.subplot(311)
plt.plot(df_resampled.index, df_resampled, label='Actual Sales')
plt.plot(test_dates, arima_forecast, label='ARIMA Forecast', color='red')
plt.title('ARIMA Forecast')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()

# Plot Prophet
plt.subplot(312)
plt.plot(df_resampled.index, df_resampled, label='Actual Sales')
plt.plot(test_dates, prophet_forecast, label='Prophet Forecast', color='green')
plt.title('Prophet Forecast')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()

# Plot LSTM
plt.subplot(313)
plt.plot(df_resampled.index, df_resampled, label='Actual Sales')
plt.plot(test_dates, lstm_forecast, label='LSTM Forecast', color='blue')
plt.title('LSTM Forecast')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()

plt.tight_layout()
plt.show()
